In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

import os
os.chdir("/gdrive/My Drive/VBO/4_hafta/")

Mounted at /gdrive/


In [ ]:
import pandas as pd
import numpy as np

##### Part 1 #####
# From first hw #

def create_movie_df(nr_drop_movie):
  #change these paths with your own path of database
    movie = pd.read_csv('/gdrive/MyDrive/VBO/4_hafta/Ders Öncesi Notlar/movie_lens_dataset/movie.csv')
    rating = pd.read_csv('/gdrive/MyDrive/VBO/4_hafta/Ders Öncesi Notlar/movie_lens_dataset/rating.csv')
    #merge datasets
    df = movie.merge(rating, how="left", on="movieId")
    rating_count = pd.DataFrame(df["title"].value_counts())
  #nr_drop_movies= number of dropped movies from list
    rare_movies = rating_count[rating_count["title"] <= nr_drop_movie].index
    common_movies = df[~df["title"].isin(rare_movies)]
    movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return movie_df

movie_df = create_movie_df(nr_drop_movie=10000)

random_user = int(pd.Series(movie_df.index).sample(1, random_state=1).values)

##### Part 2 #####

random_user_df = movie_df[movie_df.index == random_user]

movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()


##### Part 3 #####

movies_watched_df = movie_df[movies_watched]
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
users_same_movies = user_movie_count[user_movie_count["movie_count"] > len(movies_watched)*0.6]["userId"]
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

users_same_movies.head()

##### Part 4 #####
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()

corr_df.head()


movie = pd.read_csv('/gdrive/MyDrive/VBO/4_hafta/Ders Öncesi Notlar/movie_lens_dataset/movie.csv')
rating = pd.read_csv('/gdrive/MyDrive/VBO/4_hafta/Ders Öncesi Notlar/movie_lens_dataset/rating.csv')

top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr', ascending=False)

top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')

top_users_ratings.head(50)

top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]


#############################################
# Adım 5: Weighted Average Recommendation Score'un Hesaplanması
#############################################

top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
top_users_ratings.head()

recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df.head()


movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)
movies_to_be_recommend.merge(movie[["movieId", "title"]])


##### Part 6 ####

movie_id=rating[(rating["userId"]==random_user)&(rating["rating"]==5)].sort_values("timestamp",ascending=False)["movieId"][0:1].values[0]
#movie_title=movie[movie["movieId"]==movie_id]["title"].values[0]

def item_based_recommender(movie_name, user_movie_df):
    movie_name = movie_df[movie_name]
    return movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)


######### #######
item_based = item_based_recommender(movie[movie["movieId"] == movie_id]["title"].values[0], user_movie_df)[1:6]
user_based = movies_to_be_recommend[1:6]





KeyError: ignored